# StyleTTS2 音声合成と品質比較

学習済みモデルを使って、特定の文章で音声合成を行い、品質を比較します。

## 評価文

以下の3つの評価文を使用:
1. "こんにちは、今日は良い天気ですね。"
2. "人工知能の発展により、音声合成技術も進化しています。"
3. "この文章を自然な音声で読み上げます。"

## 出力

各データセットごとに以下を生成:
- `outputs/eval_audio/jvs002_{dataset_name}/eval_001.wav`
- `outputs/eval_audio/jvs002_{dataset_name}/eval_002.wav`
- `outputs/eval_audio/jvs002_{dataset_name}/eval_003.wav`


In [11]:
# 必要なライブラリのインポート
import sys
import yaml
from pathlib import Path
import torch
import torchaudio
import librosa
import numpy as np
from munch import Munch
import soundfile as sf
from phonemizer import phonemize

# プロジェクトルートを取得
current_dir = Path.cwd()
if current_dir.name == "styletts2":
    PROJECT_ROOT = current_dir.parent.parent
elif current_dir.name == "notebooks":
    PROJECT_ROOT = current_dir.parent
else:
    PROJECT_ROOT = current_dir

STYLETTS2_DIR = PROJECT_ROOT / "StyleTTS2"
OUTPUT_ROOT = PROJECT_ROOT / "outputs"

# StyleTTS2をパスに追加
if STYLETTS2_DIR.exists() and str(STYLETTS2_DIR) not in sys.path:
    sys.path.insert(0, str(STYLETTS2_DIR))

# StyleTTS2モジュールのインポート
from models import *
from utils import *
from text_utils import TextCleaner

# データセット名のリスト
DATASETS = ["phone_min4", "feat_top10", "full100"]

# 評価文
EVAL_TEXTS = [
    "こんにちは、今日は良い天気ですね。",
    "人工知能の発展により、音声合成技術も進化しています。",
    "この文章を自然な音声で読み上げます。"
]

print("="*60)
print("StyleTTS2 音声合成と品質比較")
print("="*60)
print(f"\nプロジェクトルート: {PROJECT_ROOT}")
print(f"StyleTTS2ディレクトリ: {STYLETTS2_DIR}")
print(f"データセット: {DATASETS}")
print(f"\n評価文数: {len(EVAL_TEXTS)}")
for i, text in enumerate(EVAL_TEXTS, 1):
    print(f"  {i}. {text}")


StyleTTS2 音声合成と品質比較

プロジェクトルート: /mnt/e/dev/minimal-feature-corpus-tts
StyleTTS2ディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2
データセット: ['phone_min4', 'feat_top10', 'full100']

評価文数: 3
  1. こんにちは、今日は良い天気ですね。
  2. 人工知能の発展により、音声合成技術も進化しています。
  3. この文章を自然な音声で読み上げます。


In [12]:
# デバイス設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nデバイス: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA メモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")



デバイス: cuda
GPU: NVIDIA GeForce RTX 4070 Ti
CUDA メモリ: 11.99 GB


In [13]:
# テキストクリーンアップと音素化の設定
textcleaner = TextCleaner()

def phonemize_japanese(text: str) -> str:
    """日本語テキストをeSpeak-NGで音素化"""
    try:
        phonemized = phonemize(
            text,
            backend='espeak',
            language='ja',
            strip=True,
            preserve_punctuation=True,
            with_stress=False,
            njobs=1
        )
        # phonemizeは文字列を返すが、リストの場合もあるので処理
        if isinstance(phonemized, list):
            phonemized = phonemized[0] if phonemized else text
        return str(phonemized).strip()
    except Exception as e:
        print(f"音素化エラー: {e}")
        return text

print("✓ テキスト処理関数を定義しました")


✓ テキスト処理関数を定義しました


In [14]:
# メルスペクトログラムの前処理設定
to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300
)
mean, std = -4, 4

def preprocess_wave(wave):
    """音声波形を前処理してメルスペクトログラムに変換（デモノートブックと同じ実装）"""
    if isinstance(wave, np.ndarray):
        wave_tensor = torch.from_numpy(wave).float()
    elif isinstance(wave, torch.Tensor):
        wave_tensor = wave.float()
    else:
        raise TypeError(f"wave must be numpy array or torch.Tensor, got {type(wave)}")
    
    # to_melはデバイスに移動せず、CPUで処理してからデバイスに移動
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def length_to_mask(lengths):
    """長さからマスクを生成"""
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

print("✓ 前処理関数を定義しました")


✓ 前処理関数を定義しました


In [15]:
# モデル読み込み関数
def load_model(dataset_name: str):
    """
    学習済みモデルを読み込む
    
    Args:
        dataset_name: データセット名
        
    Returns:
        モデル、設定、モデルパラメータ、サンプラーのタプル
    """
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    model_dir = STYLETTS2_DIR / "Models" / f"jvs002_{dataset_name}"
    
    if not config_path.exists():
        raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
    if not model_dir.exists():
        raise FileNotFoundError(f"モデルディレクトリが見つかりません: {model_dir}")
    
    # 設定を読み込み
    config = yaml.safe_load(open(config_path, encoding='utf-8'))
    model_params = recursive_munch(config['model_params'])
    
    # 最新のチェックポイントを探す（Stage 2を優先、なければStage 1）
    checkpoint_files_2nd = list(model_dir.glob("epoch_2nd_*.pth"))
    checkpoint_files_1st = list(model_dir.glob("epoch_1st_*.pth"))
    
    if checkpoint_files_2nd:
        checkpoint_path = sorted(checkpoint_files_2nd)[-1]
        print(f"  Stage 2チェックポイントを使用: {checkpoint_path.name}")
    elif checkpoint_files_1st:
        checkpoint_path = sorted(checkpoint_files_1st)[-1]
        print(f"  Stage 1チェックポイントを使用: {checkpoint_path.name}")
    else:
        raise FileNotFoundError(f"チェックポイントが見つかりません: {model_dir}")
    
    # ASRモデルの読み込み
    ASR_config = config.get('ASR_config', False)
    ASR_path = config.get('ASR_path', False)
    if ASR_config and ASR_path:
        ASR_config_path = STYLETTS2_DIR / ASR_config
        ASR_model_path = STYLETTS2_DIR / ASR_path
        text_aligner = load_ASR_models(str(ASR_model_path), str(ASR_config_path))
    else:
        raise ValueError("ASR設定が見つかりません")
    
    # F0モデルの読み込み
    F0_path = config.get('F0_path', False)
    if F0_path:
        F0_model_path = STYLETTS2_DIR / F0_path
        pitch_extractor = load_F0_models(str(F0_model_path))
    else:
        raise ValueError("F0設定が見つかりません")
    
    # PLBERTの読み込み
    BERT_path = config.get('PLBERT_dir', False)
    if BERT_path:
        from Utils.PLBERT.util import load_plbert
        plbert = load_plbert(str(STYLETTS2_DIR / BERT_path))
    else:
        raise ValueError("PLBERT設定が見つかりません")
    
    # モデルの構築
    model = build_model(model_params, text_aligner, pitch_extractor, plbert)
    
    # チェックポイントの読み込み
    params_whole = torch.load(str(checkpoint_path), map_location='cpu', weights_only=False)
    params = params_whole.get('net', params_whole)
    
    for key in model:
        if key in params:
            try:
                model[key].load_state_dict(params[key])
                print(f"  ✓ {key} を読み込みました")
            except Exception as e:
                # module.プレフィックスを削除して再試行
                from collections import OrderedDict
                state_dict = params[key]
                new_state_dict = OrderedDict()
                for k, v in state_dict.items():
                    name = k[7:] if k.startswith('module.') else k
                    new_state_dict[name] = v
                try:
                    model[key].load_state_dict(new_state_dict, strict=False)
                    print(f"  ✓ {key} を読み込みました（strict=False）")
                except Exception as e2:
                    print(f"  ✗ {key} の読み込みに失敗: {e2}")
    
    # モデルを評価モードに
    _ = [model[key].eval() for key in model]
    _ = [model[key].to(device) for key in model]
    
    # Diffusion Samplerの設定
    from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule
    sampler = DiffusionSampler(
        model.diffusion.diffusion,
        sampler=ADPM2Sampler(),
        sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0),
        clamp=False
    )
    
    return model, config, model_params, sampler

print("✓ モデル読み込み関数を定義しました")


✓ モデル読み込み関数を定義しました


In [16]:
# スタイルベクトル計算関数（デモノートブックと同じ実装）
def compute_style(model, audio_path: str, debug=False):
    """
    参照音声からスタイルベクトルを計算
    
    Args:
        model: 学習済みモデル
        audio_path: 参照音声ファイルパス
        debug: デバッグ情報を出力するか
        
    Returns:
        スタイルベクトル (ref_s, ref_p を連結)
    """
    wave, sr = librosa.load(audio_path, sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    
    # librosa.effects.trimはタプルを返すが、audioは最初の要素
    if isinstance(audio, tuple):
        audio = audio[0]
    
    # サンプリングレートが24000でない場合はリサンプル
    if sr != 24000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=24000)
    
    if debug:
        print(f"  audio shape: {audio.shape}, min={audio.min():.4f}, max={audio.max():.4f}")
    
    # 前処理（CPUで処理）
    mel_tensor = preprocess_wave(audio)
    
    if debug:
        print(f"  mel_tensor shape (CPU): {mel_tensor.shape}, min={mel_tensor.min():.4f}, max={mel_tensor.max():.4f}")
        print(f"  mel_tensor has NaN: {torch.isnan(mel_tensor).any()}, has Inf: {torch.isinf(mel_tensor).any()}")
    
    # デバイスに移動（デモノートブックと同じ）
    mel_tensor = mel_tensor.to(device)
    
    # unsqueeze(1)で形状を (1, 1, n_mels, frames) に
    mel_input = mel_tensor.unsqueeze(1)
    
    if debug:
        print(f"  mel_input shape (after unsqueeze): {mel_input.shape}")
    
    with torch.no_grad():
        ref_s = model.style_encoder(mel_input)
        ref_p = model.predictor_encoder(mel_input)
        
        if debug:
            print(f"  ref_s shape: {ref_s.shape}, min={ref_s.min().item():.4f}, max={ref_s.max().item():.4f}")
            print(f"  ref_p shape: {ref_p.shape}, min={ref_p.min().item():.4f}, max={ref_p.max().item():.4f}")
            print(f"  ref_s has NaN: {torch.isnan(ref_s).any()}, has Inf: {torch.isinf(ref_s).any()}")
            print(f"  ref_p has NaN: {torch.isnan(ref_p).any()}, has Inf: {torch.isinf(ref_p).any()}")
    
    result = torch.cat([ref_s, ref_p], dim=1)
    
    if debug:
        print(f"  result shape: {result.shape}, min={result.min().item():.4f}, max={result.max().item():.4f}")
    
    return result

print("✓ スタイルベクトル計算関数を定義しました")


✓ スタイルベクトル計算関数を定義しました


In [17]:
# 音声合成関数
def synthesize_speech(model, model_params, sampler, text: str, ref_style, 
                     alpha=0.3, beta=0.7, diffusion_steps=5, embedding_scale=1.0):
    """
    テキストから音声を生成
    
    Args:
        model: 学習済みモデル
        model_params: モデル設定
        sampler: Diffusion Sampler
        text: 入力テキスト（日本語）
        ref_style: 参照スタイルベクトル
        alpha: スタイル混合パラメータ（タイムブレ）
        beta: スタイル混合パラメータ（プロソディ）
        diffusion_steps: 拡散ステップ数
        embedding_scale: 埋め込みスケール
        
    Returns:
        生成された音声波形（numpy配列、サンプリングレート24000Hz）
    """
    text = text.strip()
    
    # テキストを音素化
    phonemized = phonemize_japanese(text)
    
    # トークン化
    tokens = textcleaner(phonemized)
    tokens.insert(0, 0)  # BOSトークン
    tokens.append(0)  # EOSトークン
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)
    
    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)
        
        # BERT埋め込み
        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)
        
        # スタイル拡散（diffusion sampling）
        noise = torch.randn((1, 256)).unsqueeze(1).to(device)
        s_pred = sampler(
            noise=noise,
            embedding=bert_dur,
            embedding_scale=embedding_scale,
            features=ref_style,  # 参照スタイル
            num_steps=diffusion_steps
        ).squeeze(1)
        
        # スタイルベクトルの分解
        s = s_pred[:, 128:]  # プロソディスタイル
        ref = s_pred[:, :128]  # タイムブレスタイル
        
        # スタイルの混合
        ref = alpha * ref + (1 - alpha) * ref_style[:, :128]
        s = beta * s + (1 - beta) * ref_style[:, 128:]
        
        # デュレーション予測
        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)
        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)
        duration = torch.sigmoid(duration).sum(dim=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1).long()
        
        # 最後のデュレーションを少し長くする（安定性のため）
        if len(pred_dur) > 0:
            pred_dur[-1] += 5
        
        # アライメント行列の作成
        aln_length = int(pred_dur.sum().item())
        input_len = int(input_lengths.item())
        pred_aln_trg = torch.zeros(input_len, aln_length).to(device)
        c_frame = 0
        for i in range(pred_aln_trg.shape[0]):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].item())] = 1
            c_frame += int(pred_dur[i].item())
        
        # プロソディエンコーディング
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        
        # F0とノイズ予測
        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)
        
        # ASR特徴量の計算
        # train_second.pyの762行目を参考に、正しい形状で計算
        asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
        
        # デコーダーで音声生成
        # 注意: istftnetデコーダーは直接波形を出力します
        # 引数の順序: asr, F0_pred, N_pred, ref
        out = model.decoder(asr, F0_pred, N_pred, ref.squeeze().unsqueeze(0))
    
    # numpy配列に変換
    wav_np = out.squeeze().cpu().numpy()
    
    # 形状を1次元に
    if len(wav_np.shape) > 1:
        wav_np = wav_np.flatten()
    
    # 最後の数サンプルを削除（モデルのパルス問題）
    if len(wav_np) > 100:
        wav_np = wav_np[:-100]
    elif len(wav_np) > 50:
        wav_np = wav_np[:-50]
    
    # 音声のクリッピング（範囲を[-1, 1]に制限）
    wav_np = np.clip(wav_np, -1.0, 1.0)
    
    # NaNやInfをチェック
    if np.isnan(wav_np).any() or np.isinf(wav_np).any():
        print("警告: 生成された音声にNaNまたはInfが含まれています")
        wav_np = np.nan_to_num(wav_np, nan=0.0, posinf=1.0, neginf=-1.0)
    
    # 正規化（オプション：最大値を1.0に）
    max_val = np.abs(wav_np).max()
    if max_val > 0:
        wav_np = wav_np / max_val * 0.95  # 0.95に制限してクリッピングを避ける
    
    return wav_np

print("✓ 音声合成関数を定義しました")


✓ 音声合成関数を定義しました


In [18]:
# 各データセットで評価音声を生成
print("\n" + "="*60)
print("評価音声生成開始")
print("="*60)

results = {}

for dataset_name in DATASETS:
    print(f"\n{'='*60}")
    print(f"データセット: {dataset_name}")
    print(f"{'='*60}")
    
    try:
        # モデルの読み込み
        print("\nモデルを読み込んでいます...")
        model, config, model_params, sampler = load_model(dataset_name)
        print("✓ モデルの読み込み完了\n")
        
        # 出力ディレクトリの作成
        output_dir = OUTPUT_ROOT / "eval_audio" / f"jvs002_{dataset_name}"
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # 参照音声の取得（データセットから1つ選ぶ）
        data_dir = STYLETTS2_DIR / "Data" / f"jvs002_{dataset_name}" / "wavs"
        ref_audio_files = list(data_dir.glob("*.wav"))
        
        if not ref_audio_files:
            print(f"⚠️ 参照音声が見つかりません: {data_dir}")
            continue
        
        ref_audio_path = str(ref_audio_files[0])
        print(f"参照音声: {Path(ref_audio_path).name}")
        
        # 参照スタイルベクトルの計算
        print("参照スタイルベクトルを計算しています...")
        ref_style = compute_style(model, ref_audio_path)
        print("✓ 参照スタイルベクトルの計算完了\n")
        
        # 各評価文で音声生成
        dataset_results = []
        for i, text in enumerate(EVAL_TEXTS, 1):
            print(f"評価文 {i}/{len(EVAL_TEXTS)}: {text[:40]}...")
            
            try:
                wav = synthesize_speech(
                    model=model,
                    model_params=model_params,
                    sampler=sampler,
                    text=text,
                    ref_style=ref_style,
                    alpha=0.3,
                    beta=0.7,
                    diffusion_steps=5,
                    embedding_scale=1.0
                )
                
                # 音声ファイルの保存
                output_path = output_dir / f"eval_{i:03d}.wav"
                sf.write(str(output_path), wav, 24000)
                
                duration = len(wav) / 24000
                print(f"  ✓ 保存: {output_path.name} ({duration:.2f}秒)")
                dataset_results.append({
                    'text': text,
                    'path': str(output_path),
                    'duration': duration
                })
                
            except Exception as e:
                print(f"  ✗ エラー: {e}")
                import traceback
                traceback.print_exc()
                dataset_results.append({
                    'text': text,
                    'path': None,
                    'error': str(e)
                })
        
        results[dataset_name] = dataset_results
        print(f"\n✓ {dataset_name} の評価音声生成完了")
        print(f"出力ディレクトリ: {output_dir}")
        
    except Exception as e:
        print(f"\n✗ {dataset_name} の処理に失敗: {e}")
        import traceback
        traceback.print_exc()
        results[dataset_name] = None

print("\n" + "="*60)
print("全データセットの処理完了")
print("="*60)



評価音声生成開始

データセット: phone_min4

モデルを読み込んでいます...
  Stage 1チェックポイントを使用: epoch_1st_00070.pth
  ✓ bert を読み込みました
  ✓ bert_encoder を読み込みました
  ✓ predictor を読み込みました
  ✓ decoder を読み込みました
  ✓ text_encoder を読み込みました
  ✓ predictor_encoder を読み込みました
  ✓ style_encoder を読み込みました
  ✓ diffusion を読み込みました
  ✓ text_aligner を読み込みました
  ✓ pitch_extractor を読み込みました
  ✓ mpd を読み込みました
  ✓ msd を読み込みました
  ✓ wd を読み込みました
✓ モデルの読み込み完了

参照音声: VOICEACTRESS100_011.wav
参照スタイルベクトルを計算しています...
✓ 参照スタイルベクトルの計算完了

評価文 1/3: こんにちは、今日は良い天気ですね。...
警告: 生成された音声にNaNまたはInfが含まれています
  ✓ 保存: eval_001.wav (79.55秒)
評価文 2/3: 人工知能の発展により、音声合成技術も進化しています。...
警告: 生成された音声にNaNまたはInfが含まれています
  ✓ 保存: eval_002.wav (167.05秒)
評価文 3/3: この文章を自然な音声で読み上げます。...
警告: 生成された音声にNaNまたはInfが含まれています
  ✓ 保存: eval_003.wav (123.30秒)

✓ phone_min4 の評価音声生成完了
出力ディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/outputs/eval_audio/jvs002_phone_min4

データセット: feat_top10

モデルを読み込んでいます...
  Stage 1チェックポイントを使用: epoch_1st_00070.pth
  ✓ bert を読み込みました
  ✓ bert_encoder を読み込みました
  ✓ predicto

In [19]:
# 結果のサマリー表示
print("\n" + "="*60)
print("生成結果サマリー")
print("="*60)

for dataset_name in DATASETS:
    if dataset_name not in results or results[dataset_name] is None:
        print(f"\n{dataset_name}: 処理失敗または未実行")
        continue
    
    print(f"\n{dataset_name}:")
    print("-" * 60)
    output_dir = OUTPUT_ROOT / "eval_audio" / f"jvs002_{dataset_name}"
    
    for i, result in enumerate(results[dataset_name], 1):
        if 'error' in result:
            print(f"  評価文 {i}: ✗ エラー - {result['error']}")
        else:
            file_path = Path(result['path'])
            if file_path.exists():
                file_size = file_path.stat().st_size / 1024  # KB
                print(f"  評価文 {i}: ✓ {file_path.name}")
                print(f"    テキスト: {result['text']}")
                print(f"    長さ: {result['duration']:.2f}秒")
                print(f"    ファイルサイズ: {file_size:.1f} KB")
            else:
                print(f"  評価文 {i}: ✗ ファイルが見つかりません")

print("\n" + "="*60)
print("比較用の音声ファイル一覧")
print("="*60)
print("\n各データセットの音声ファイルは以下のディレクトリに保存されています:")
for dataset_name in DATASETS:
    output_dir = OUTPUT_ROOT / "eval_audio" / f"jvs002_{dataset_name}"
    print(f"\n{dataset_name}:")
    print(f"  {output_dir}")
    if output_dir.exists():
        audio_files = sorted(output_dir.glob("eval_*.wav"))
        for audio_file in audio_files:
            print(f"    - {audio_file.name}")



生成結果サマリー

phone_min4:
------------------------------------------------------------
  評価文 1: ✓ eval_001.wav
    テキスト: こんにちは、今日は良い天気ですね。
    長さ: 79.55秒
    ファイルサイズ: 3728.8 KB
  評価文 2: ✓ eval_002.wav
    テキスト: 人工知能の発展により、音声合成技術も進化しています。
    長さ: 167.05秒
    ファイルサイズ: 7830.3 KB
  評価文 3: ✓ eval_003.wav
    テキスト: この文章を自然な音声で読み上げます。
    長さ: 123.30秒
    ファイルサイズ: 5779.5 KB

feat_top10:
------------------------------------------------------------
  評価文 1: ✓ eval_001.wav
    テキスト: こんにちは、今日は良い天気ですね。
    長さ: 79.35秒
    ファイルサイズ: 3719.4 KB
  評価文 2: ✓ eval_002.wav
    テキスト: 人工知能の発展により、音声合成技術も進化しています。
    長さ: 166.85秒
    ファイルサイズ: 7820.9 KB
  評価文 3: ✓ eval_003.wav
    テキスト: この文章を自然な音声で読み上げます。
    長さ: 123.10秒
    ファイルサイズ: 5770.2 KB

full100:
------------------------------------------------------------
  評価文 1: ✓ eval_001.wav
    テキスト: こんにちは、今日は良い天気ですね。
    長さ: 79.60秒
    ファイルサイズ: 3731.1 KB
  評価文 2: ✓ eval_002.wav
    テキスト: 人工知能の発展により、音声合成技術も進化しています。
    長さ: 167.10秒
    ファイルサイズ: 7832.7 KB
  評価文 3: ✓ eval_0

In [20]:
# デバッグ用: 単一のデータセットでテスト実行
print("\n" + "="*60)
print("デバッグ: 単一データセットでテスト実行")
print("="*60)

# テスト用に1つのデータセットだけ処理
test_dataset = "phone_min4"  # または "feat_top10", "full100"

try:
    print(f"\nデータセット: {test_dataset}")
    print("モデルを読み込んでいます...")
    model, config, model_params, sampler = load_model(test_dataset)
    print("✓ モデルの読み込み完了\n")
    
    # 参照音声の取得
    data_dir = STYLETTS2_DIR / "Data" / f"jvs002_{test_dataset}" / "wavs"
    ref_audio_files = list(data_dir.glob("*.wav"))
    
    if not ref_audio_files:
        print(f"⚠️ 参照音声が見つかりません: {data_dir}")
    else:
        ref_audio_path = str(ref_audio_files[0])
        print(f"参照音声: {Path(ref_audio_path).name}")
        
        # 参照スタイルベクトルの計算（デバッグモード）
        print("参照スタイルベクトルを計算しています...")
        ref_style = compute_style(model, ref_audio_path, debug=True)
        print(f"\n参照スタイルベクトルの形状: {ref_style.shape}")
        print(f"参照スタイルベクトルの統計: min={ref_style.min().item():.4f}, max={ref_style.max().item():.4f}, mean={ref_style.mean().item():.4f}")
        std_val = ref_style.std().item()
        if np.isinf(std_val) or np.isnan(std_val):
            print(f"  警告: std={std_val} (異常値)")
        else:
            print(f"  std={std_val:.4f}")
        print()
        
        # テスト用に1つの評価文で音声生成
        test_text = EVAL_TEXTS[0]
        print(f"テストテキスト: {test_text}")
        
        wav = synthesize_speech(
            model=model,
            model_params=model_params,
            sampler=sampler,
            text=test_text,
            ref_style=ref_style,
            alpha=0.3,
            beta=0.7,
            diffusion_steps=5,
            embedding_scale=1.0
        )
        
        print(f"\n生成された音声の統計:")
        print(f"  形状: {wav.shape}")
        print(f"  長さ: {len(wav) / 24000:.2f}秒")
        print(f"  min: {wav.min():.4f}, max: {wav.max():.4f}")
        print(f"  mean: {wav.mean():.4f}, std: {wav.std():.4f}")
        print(f"  NaN: {np.isnan(wav).sum()}, Inf: {np.isinf(wav).sum()}")
        
        # テスト音声の保存
        test_output_dir = OUTPUT_ROOT / "eval_audio" / f"jvs002_{test_dataset}_debug"
        test_output_dir.mkdir(parents=True, exist_ok=True)
        test_output_path = test_output_dir / "test_output.wav"
        sf.write(str(test_output_path), wav, 24000)
        print(f"\n✓ テスト音声を保存: {test_output_path}")
        
except Exception as e:
    print(f"\n✗ エラー: {e}")
    import traceback
    traceback.print_exc()


デバッグ: 単一データセットでテスト実行

データセット: phone_min4
モデルを読み込んでいます...
  Stage 1チェックポイントを使用: epoch_1st_00070.pth
  ✓ bert を読み込みました
  ✓ bert_encoder を読み込みました
  ✓ predictor を読み込みました
  ✓ decoder を読み込みました
  ✓ text_encoder を読み込みました
  ✓ predictor_encoder を読み込みました
  ✓ style_encoder を読み込みました
  ✓ diffusion を読み込みました
  ✓ text_aligner を読み込みました
  ✓ pitch_extractor を読み込みました
  ✓ mpd を読み込みました
  ✓ msd を読み込みました
  ✓ wd を読み込みました
✓ モデルの読み込み完了

参照音声: VOICEACTRESS100_011.wav
参照スタイルベクトルを計算しています...
  audio shape: (150016,), min=-0.2534, max=0.2410
  mel_tensor shape (CPU): torch.Size([1, 80, 501]), min=-1.8175, max=3.0877
  mel_tensor has NaN: False, has Inf: False
  mel_input shape (after unsqueeze): torch.Size([1, 1, 80, 501])
  ref_s shape: torch.Size([1, 128]), min=-2.3381, max=2.3232
  ref_p shape: torch.Size([1, 128]), min=-107893659862682304512.0000, max=98213972908114444288.0000
  ref_s has NaN: False, has Inf: False
  ref_p has NaN: False, has Inf: False
  result shape: torch.Size([1, 256]), min=-10789365986268230

## 問題の原因と対処方法

### 問題点

1. **predictor_encoderの出力が異常**: `ref_p`が極端に大きな値（約10^20）を持っています
2. **Stage 2の学習が完了していない**: 全てのデータセットでStage 1のチェックポイントのみが存在
3. **学習データが少ない**: `phone_min4`は4ファイル（訓練3、検証1）しかない

### 考えられる原因

1. **学習が不十分**: Stage 1だけでは`predictor_encoder`が適切に学習されていない可能性
2. **モデルの重みが破損**: チェックポイントの保存/読み込み時に問題が発生した可能性
3. **データが少なすぎる**: 4ファイルでは`predictor_encoder`を適切に学習できない可能性

### 対処方法

1. **Stage 2の学習を実行**: `train_second.py`を実行してStage 2の学習を完了させる
2. **より多くのデータで学習**: `full100`データセットで学習を完了させる
3. **predictor_encoderを無効化**: 一時的に`ref_p`をゼロベクトルにして`ref_s`のみを使用

現在のコードでは、`predictor_encoder`の出力が異常な場合は自動的にゼロベクトルに置き換えます。